### Data Comparison.

1. This notebook illustrates how to compare data values with matching timestamps for science variables from two data delivery methods of a CTD (Conductivity Temperature Depth) sensor deployed on a Global Papa Station in the Northern Pacific Ocean (sensor reference designator: GP03FLMB-RIM01-02-CTDMOG060). 

Click on the [Link](https://drive.google.com/open?id=1_kAzdVfou_PWG-UnwB7Nib4NpyktKsVg) for more information about the station.

**Missing Data Test:**
<blockquote>  Provides the number of gaps and days of data that are missing in the "preferred" dataset delivery method and are available in a "non-preferred" dataset delivery method. </blockquote> 

**Data Comparison:** 
<blockquote> Compare data values with matching timestamps for science variables among all delivery methods.</blockquote> 

We will be using two keywords for data delivery methods in this example:
- <font color="red"> recovered: </font>  data collected after the instrument was recovered from the water. This is the preferred data delivery method because it is expected to be the most complete.

- <font color="red"> telemetered: </font> data transmitted via a telemetry system while the instrument is in the water. This is the non-preferred data delivery method because it is less complete (not all data are transmitted via telemetry. The files are subsampled for the telemetry system to work).

### Outline:
- [Python Packages.](#1)
- [Data File.](#2)
    - [Telemetered Data Files.](#21)
    - [ Recovered Data Files.](#22)
- [Define Functions.](#3)
- [Data Comparison.](#4)
    - [Selected Datasets for Comparison.](#41)
    - [Load recovered Dataset.](#42)
    - [Load telemetered Dataset.](#43)
    - [Map Parameters from the Two Files.](#44)
    - [View files data comparison result:](#45)
- [Observations.](#5)
- [Summary.](#6)

<span style='color:Orange' size=20 > **Attention:** </span> 
- To run the notebook, you need to follow the septs in order.
- For the code cell, run the cell before you move on to the next one. 
    - **Remember**: The output of a cell may be an input in the next cell.

<a id="1" ></a>
### Python Packages.

In [1]:
import xarray as xr
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import requests
from datetime import timedelta

<a id="2" ></a>
### Data File.

- Get the list of files from all data delivery methods. Two .csv files are used for this example:

         *_recovered.csv   &  *_telemetered.csv

In [2]:
%cd '/Users/leilabelabassi/Desktop/TAMU/online-class/612-DataQuality4theGeosciences/class_material/Module4_csvFiles/'

file_recovered = 'data_review_list_GP03FLMB-RIM01-02-CTDMOG060_recovered.csv'
list_recovered = pd.read_csv(file_recovered)

file_telemetered = 'data_review_list_GP03FLMB-RIM01-02-CTDMOG060_telemetered.csv'
list_telemetered = pd.read_csv(file_telemetered)


/Users/leilabelabassi/Desktop/TAMU/online-class/612-DataQuality4theGeosciences/class_material/Module4_csvFiles


<a id="21" ></a>
#### Telemetered Data Files.

In [3]:
# Print the list of files for the telemetered data delivery method
pd.set_option('display.max_colwidth', None)
(list_telemetered)

,files
0,deployment0001_GP03FLMB-RIM01-02-CTDMOG060-telemetered-ctdmo_ghqr_sio_mule_instrument_20130724T100001-20140227T140001.nc
1,deployment0002_GP03FLMB-RIM01-02-CTDMOG060-telemetered-ctdmo_ghqr_sio_mule_instrument_20140620T040001-20141109T000001.nc
2,deployment0003_GP03FLMB-RIM01-02-CTDMOG060-telemetered-ctdmo_ghqr_sio_mule_instrument_20150609T000001-20160209T220001.nc
3,deployment0004_GP03FLMB-RIM01-02-CTDMOG060-telemetered-ctdmo_ghqr_sio_mule_instrument_20161008T080001-20161219T000001.nc
4,deployment0007_GP03FLMB-RIM01-02-CTDMOG060-telemetered-ctdmo_ghqr_sio_mule_instrument_20190928T000001-20200118T200001.nc


<a id="22" ></a>
#### Recovered Data Files.

In [4]:
# Print the list of files for the recovered data delivery method
pd.set_option('display.max_colwidth', None)
list_recovered

,files
0,deployment0001_GP03FLMB-RIM01-02-CTDMOG060-recovered_inst-ctdmo_ghqr_instrument_recovered_20130724T064501-20140617T234501.nc
1,deployment0003_GP03FLMB-RIM01-02-CTDMOG060-recovered_inst-ctdmo_ghqr_instrument_recovered_20150608T213001-20160703T183001.nc
2,deployment0004_GP03FLMB-RIM01-02-CTDMOG060-recovered_inst-ctdmo_ghqr_instrument_recovered_20160704T231501-20170717T150001.nc
3,deployment0005_GP03FLMB-RIM01-02-CTDMOG060-recovered_inst-ctdmo_ghqr_instrument_recovered_20170714T230001-20180725T170001.nc
4,deployment0006_GP03FLMB-RIM01-02-CTDMOG060-recovered_inst-ctdmo_ghqr_instrument_recovered_20180724T231501-20190927T234501.nc


<a id="3" ></a>
### Define Functions.
- The functions are created to divide the process into defined steps that can be called separately to execute a specific data quality module.  

In [5]:
def return_science_vars(stream):
    """
    Returns data parameters with scientific units relevent ocean processes.
    Source URL:
    http://datareview.marine.rutgers.edu/streams/view/ctdmo_ghqr_instrument_recovered.json
    """
    sci_vars = []
    dr = 'http://datareview.marine.rutgers.edu/streams/view/{}.json'.format(stream)
    r = requests.get(dr)
    params = r.json()['stream']['parameters']
    for p in params:
        if p['data_product_type'] == 'Science Data':
            sci_vars.append(p['name'])
    return sci_vars

In [6]:
def long_names(dataset, vars):
    """
    Returns the long names for the science data parameters.
    Long names are used to match parameters from two delivery methods. 
    """
    name = []
    long_name = []
    for v in vars:
        name.append(v)
        try:
            longname = dataset[v].long_name
        except AttributeError:
            longname = vars
        long_name.append(longname)

    return pd.DataFrame({'name': name, 'long_name': long_name})

In [7]:
def var_units(variable):
    """
    Returns units for science data parameters.
    """
    try:
        y_units = variable.units
    except AttributeError:
        y_units = 'no_units'

    return y_units

In [8]:
def get_ds_variable_info(dataset, variable_name, rename):
    """
    Returns the parameter's data and time array.
    Returns the parameter's unit using var_units function.
    Returns the count of nans in the parameter's data array.
    """
    ds_units = var_units(dataset[variable_name])

    ds_df = pd.DataFrame({'time': dataset['time'].values, variable_name: dataset[variable_name].values})
    ds_df.rename(columns={str(variable_name): rename}, inplace=True)
    n = len(ds_df[rename])
    n_nan = sum(ds_df[rename].isnull())

    # round to the nearest second
    ds_df['time'] = ds_df['time'].map(lambda t: t.replace(microsecond=0) + timedelta(seconds=(round(t.microsecond / 1000000.0))))

    return [ds_df, ds_units, n, n_nan]

In [9]:
def missing_data_times(df):
    '''
    Compares two delivery methods and returns a dictionary with information about missing data:
    - time ranges   - number of data points   - number of days.
    Skips gaps that are only 1 data point (or one hour if data are rounded to the hour).
    '''
   
    md_list = []
    n_list = []
    mdays = []
    index_break = []
    ilist = df.index.tolist()

    if len(ilist) == 1:
        ii = ilist[0]
        md_list.append(pd.to_datetime(str(df['time'][ii])).strftime('%Y-%m-%dT%H:%M:%S'))
        n_list.append(1)
    else:
        for i, n in enumerate(ilist):
            if i == 0:
                index_break.append(ilist[i])
            elif i == (len(ilist) - 1):
                index_break.append(ilist[i])
            else:
                if (n - ilist[i-1]) > 1:
                    index_break.append(ilist[i-1])
                    index_break.append(ilist[i])

        for ii, nn in enumerate(index_break):
            if ii % 2 == 0:  # check that the index is an even number
                if index_break[ii + 1] != nn:  # only list gaps that are more than 1 data point
                    try:
                        # create a list of timestamps for each gap to get the unique # of days missing from one dataset
                        time_lst = [df['time'][t].date() for t in range(nn, index_break[ii + 1] + 1)]
                    except KeyError:  # if the last data gap is only 1 point, skip
                        continue
                    md_list.append([pd.to_datetime(str(df['time'][nn])).strftime('%Y-%m-%dT%H:%M:%S'),
                                    pd.to_datetime(str(df['time'][index_break[ii + 1]])).strftime('%Y-%m-%dT%H:%M:%S')])
                    n_list.append(index_break[ii + 1] - nn + 1)
                    mdays.append(len(np.unique(time_lst)))

    n_total = sum(n_list)
    n_days = sum(mdays)

    return dict(missing_data_gaps=md_list, n_missing=n_list, n_missing_total=n_total, n_missing_days_total=n_days)

In [10]:
def unique_timestamps_hour(ds):
    # returns dataframe of unique timestamps rounded to the nearest hour
    df = pd.DataFrame(ds['time'].values, columns=['time'])
    df = df['time'].map(lambda t: t.replace(second=0, microsecond=0, nanosecond=0, minute=0, hour=t.hour) + timedelta(hours=t.minute // 30))
    udf = pd.DataFrame(np.unique(df), columns=['time'])

    return udf

#### <font color="red"> Attention: </font> The data comparison process is created as a function (**compare_datasets**) so it can be used with any two data files.

In [11]:
def compare_datasets(mapping, preferred_method, deployment, ds0, method0, ds1, method1):
    '''
    returns parameter's unit_test, number of nan, missing data status, 
    number of data points compared,
    min and max of absolute differences,
    number of data points with a difference greater than 0.9999999999999999, 
    percent of data points with a difference greater than 0.9999999999999999.
    '''
    
    # Initialize Variables
    df =  pd.DataFrame()
    missing_data_list = []
    diff_gzero_list = []
    var_list = []
    blank_dict = {'missing_data_gaps': [], 'n_missing': [], 'n_missing_days_total': 0,
                                          'n_missing_total': 0}
    for rr in mapping.itertuples():
        index, name, long_name_x, long_name_y = rr

        ds0_rename = '_'.join((str(name), 'ds0'))
        [ds0_df, ds0_units, n0, n0_nan] = get_ds_variable_info(ds0, name, ds0_rename)
        ds1_rename = '_'.join((str(name), 'ds1'))
        [ds1_df, ds1_units, n1, n1_nan] = get_ds_variable_info(ds1, name, ds1_rename)
              
        # Compare units
        if ds0_units == ds1_units:
            unit_test = 'pass'
        else:
            unit_test = 'fail'

        # Merge dataframes from both methods
        merged = pd.merge(ds0_df, ds1_df, on='time', how='outer')

        # Drop rows where both variables are NaNs, and make sure the timestamps are in order
        merged.dropna(subset=[ds0_rename, ds1_rename], how='all', inplace=True)

        if len(merged) == 0:
            print('No valid data to compare')
            n_comparison = 0
            n_diff_g_zero = None
            min_diff = None
            max_diff = None
            ds0_missing_dict = 'No valid data to compare'
            ds1_missing_dict = 'No valid data to compare'
        else:
            merged = merged.sort_values('time').reset_index(drop=True)
            m_intersect = merged[merged[ds0_rename].notnull() & merged[ds1_rename].notnull()]

            # If the number of data points for comparison is less than 1% of the smaller sample size
            # compare the timestamps by rounding to the nearest hour
            if len(m_intersect) == 0 or float(len(m_intersect))/float(min(n0, n1))*100 < 1.00:
                n_comparison = 0
                n_diff_g_zero = None
                min_diff = None
                max_diff = None

                utime_df0 = unique_timestamps_hour(ds0)
                utime_df0['ds0'] = 'ds0'
                utime_df1 = unique_timestamps_hour(ds1)
                utime_df1['ds1'] = 'ds1'
                umerged = pd.merge(utime_df0, utime_df1, on='time', how='outer')
                umerged = umerged.sort_values('time').reset_index(drop=True)

                if 'telemetered' in method0:
                    ds0_missing_dict = 'method not checked for missing data'
                else:
                    ds0_missing = umerged.loc[umerged['ds0'].isnull()]
                    if len(ds0_missing) > 0:
                        ds0_missing_dict = missing_data_times(ds0_missing)
                        if ds0_missing_dict != blank_dict:
                            ds0_missing_dict['n_hours_missing'] = ds0_missing_dict.pop('n_missing')
                            ds0_missing_dict['n_hours_missing_total'] = ds0_missing_dict.pop('n_missing_total')
                        else:
                            ds0_missing_dict = 'timestamps rounded to the hour: no missing data'
                    else:
                        ds0_missing_dict = 'timestamps rounded to the hour: no missing data'

                if 'telemetered' in method1:
                    ds1_missing_dict = 'method not checked for missing data'
                else:
                    ds1_missing = umerged.loc[umerged['ds1'].isnull()]
                    if len(ds1_missing) > 0:
                        ds1_missing_dict = missing_data_times(ds1_missing)
                        if ds1_missing_dict != blank_dict:
                            ds1_missing_dict['n_hours_missing'] = ds1_missing_dict.pop('n_missing')
                            ds1_missing_dict['n_hours_missing_total'] = ds1_missing_dict.pop('n_missing_total')
                        else:
                            ds1_missing_dict = 'timestamps rounded to the hour: no missing data'
                    else:
                        ds1_missing_dict = 'timestamps rounded to the hour: no missing data'

            else:
                # Find where data are available in one dataset and missing in the other if
                # timestamps match exactly. Don't check for missing data in telemetered
                # datasets.
                if 'telemetered' in method0:
                    ds0_missing_dict = 'method not checked for missing data'
                else:
                    ds0_missing = merged.loc[merged[ds0_rename].isnull()]
                    if len(ds0_missing) > 0:
                        ds0_missing_dict = missing_data_times(ds0_missing)
                        if ds0_missing_dict == blank_dict:
                            ds0_missing_dict = 'no missing data'
                    else:
                        ds0_missing_dict = 'no missing data'

                if 'telemetered' in method1:
                    ds1_missing_dict = 'method not checked for missing data'
                else:
                    ds1_missing = merged.loc[merged[ds1_rename].isnull()]
                    if len(ds1_missing) > 0:
                        ds1_missing_dict = missing_data_times(ds1_missing)
                        if ds1_missing_dict == blank_dict:
                            ds1_missing_dict = 'no missing data'
                    else:
                        ds1_missing_dict = 'no missing data'

                # Where the data intersect, calculate the difference between the methods
                diff = m_intersect[ds0_rename] - m_intersect[ds1_rename]
                n_diff_g_zero = sum(abs(diff) > 0.99999999999999999)

                min_diff = round(min(abs(diff)), 10)
                max_diff = round(max(abs(diff)), 10)
                n_comparison = len(diff)
          
        compare_summary = dict( 
                                ds0=dict(name=ds0_rename, units=ds0_units, n=n0, n_nan=n0_nan, missing=ds0_missing_dict),
                                ds1=dict(name=ds1_rename, units=ds1_units, n=n1, n_nan=n1_nan, missing=ds1_missing_dict),
                                unit_test=unit_test, n_comparison=n_comparison, n_diff_greater_zero=n_diff_g_zero,
                                min_abs_diff=min_diff, max_abs_diff=max_diff
                              )
        
        name = compare_summary[preferred_method]['name']
        units = compare_summary[preferred_method]['units']
        unit_test = compare_summary['unit_test']
        n = compare_summary[preferred_method]['n']
        n_nan = compare_summary[preferred_method]['n_nan']
        missing_data = compare_summary[preferred_method]['missing']
        n_comparison = compare_summary['n_comparison']
        min_abs_diff = compare_summary['min_abs_diff']
        max_abs_diff = compare_summary['max_abs_diff']
        n_diff_greater_zero = compare_summary['n_diff_greater_zero']
        
        if n_comparison > 0:
            percent_diff_greater_zero = round((float(n_diff_greater_zero)/float(n_comparison) * 100), 2)
        else:
            percent_diff_greater_zero = None

        missing_data_list.append(str(missing_data))
        diff_gzero_list.append(percent_diff_greater_zero)
        var_list.append(name) 

        df0 = pd.DataFrame({
                            'parameter':[name], 
                            'unit': [units], 
                            'unit_test': [unit_test], 
                            'n': [n] ,
                            'n_nan': [n_nan] ,
                            'missing_data': [missing_data], 
                            'n_comparison': [n_comparison], 
                            'min_abs_diff': [min_abs_diff], 
                            'max_abs_diff': [max_abs_diff],
                            'n_diff_greater_0.99': [n_diff_greater_zero], 
                            'percent_diff_greater_zero': [percent_diff_greater_zero]},
                            index= [deployment])
        df = df.append(df0)
        
    return df

<a id="4" ></a>
### Data Comparison.

#### Selected Datasets for Comparison.
For this example, we are comparing deployment 1 telemetered and recovered data.
The attributes needed for this comparison exist in the file name:

<span style='color:blue'>Deployment number</span>: deployment**0001**_GP03FLMB-RIM01-02-CTDMOG060-recovered_inst-ctdmo_ghqr_instrument_recovered_20130724T064501-20140617T234501.nc

<span style='color:blue'>Sensor Reference Designator </span>: deployment0001_**GP03FLMB-RIM01-02-CTDMOG060**-recovered_inst-ctdmo_ghqr_instrument_recovered_20130724T064501-20140617T234501.nc

<span style='color:blue'>Data Delivery Method</span>: deployment0001_GP03FLMB-RIM01-02-CTDMOG060-**recovered_inst**-ctdmo_ghqr_instrument_recovered_20130724T064501-20140617T234501.nc

<span style='color:blue'>Data Stream</span>: deployment0001_GP03FLMB-RIM01-02-CTDMOG060-recovered_inst-**ctdmo_ghqr_instrument_recovered**_20130724T064501-20140617T234501.nc

<span style='color:blue'>Data Time Coverage</span>: deployment0001_GP03FLMB-RIM01-02-CTDMOG060-recovered_inst-ctdmo_ghqr_instrument_recovered_**20130724T064501-20140617T234501**.nc

<a id="41" ></a>
#### Load recovered Dataset.

In [12]:
%cd '/Users/leilabelabassi/Desktop/TAMU/online-class/612-DataQuality4theGeosciences/class_material/Module3_DataFiles_recovered-GP03FLMB-RIM01-02-CTDMOG060'
# Load data
ds0 = xr.open_dataset('deployment0001_GP03FLMB-RIM01-02-CTDMOG060-recovered_inst-ctdmo_ghqr_instrument_recovered_20130724T064501-20140617T234501.nc')

# data delivery method (copied from the file name):
method0 = 'recovered_inst'

# data stream for recoverd data delivery method (copied from the file name):
stream0 = 'ctdmo_ghqr_instrument_recovered'

# The preferred data recovery method for the oceanographic instrument used in this example 
# is always recovered. Telemetered data have less data. Here the string 'ds0' is used to reference to recovered data.
preferred_method = 'ds0' 
                         
# deployment number (copied from the file name):
deployment = '1'


# Get list of science parameters
sci_var0 = return_science_vars(stream0) 
long_name0 = long_names(ds0, sci_var0)


/Users/leilabelabassi/Desktop/TAMU/online-class/612-DataQuality4theGeosciences/class_material/Module3_DataFiles_recovered-GP03FLMB-RIM01-02-CTDMOG060


In [13]:
sci_var0

['density',
 'practical_salinity',
 'ctdmo_seawater_pressure',
 'ctdmo_seawater_temperature',
 'ctdmo_seawater_conductivity']

<a id="42" ></a>
#### Load telemetered Dataset.

In [14]:
%cd '/Users/leilabelabassi/Desktop/TAMU/online-class/612-DataQuality4theGeosciences/class_material/Module3_DataFiles_telemetered-GP03FLMB-RIM01-02-CTDMOG060/'

# Load data
ds1 = xr.open_dataset('deployment0001_GP03FLMB-RIM01-02-CTDMOG060-telemetered-ctdmo_ghqr_sio_mule_instrument_20130724T100001-20140227T140001.nc')
 
method1 = 'telemetered'
stream1 = 'ctdmo_ghqr_sio_mule_instrument'

# Get list of science parameters
sci_var1 = return_science_vars(stream1) 
long_name1 = long_names(ds1, sci_var1)


/Users/leilabelabassi/Desktop/TAMU/online-class/612-DataQuality4theGeosciences/class_material/Module3_DataFiles_telemetered-GP03FLMB-RIM01-02-CTDMOG060


In [15]:
long_name1

,name,long_name
0,density,Seawater Density
1,practical_salinity,Practical Salinity
2,ctdmo_seawater_pressure,Seawater Pressure
3,ctdmo_seawater_temperature,Seawater Temperature
4,ctdmo_seawater_conductivity,Seawater Conductivity


<a id="43" ></a>
#### Map Parameters from the Two Files.

In [16]:
mapping = pd.merge(long_name0, long_name1, on='name', how='inner')
mapping

,name,long_name_x,long_name_y
0,density,Seawater Density,Seawater Density
1,practical_salinity,Practical Salinity,Practical Salinity
2,ctdmo_seawater_pressure,Seawater Pressure,Seawater Pressure
3,ctdmo_seawater_temperature,Seawater Temperature,Seawater Temperature
4,ctdmo_seawater_conductivity,Seawater Conductivity,Seawater Conductivity


<a id="44" ></a>
#### Run Data Comparison for the Slected Files.

In [17]:
# Call function and run it.
df = compare_datasets(mapping, preferred_method, deployment, ds0, method0, ds1, method1)

<a id="45" ></a>
#### View files data comparison result:

In [18]:
pd.set_option('display.max_colwidth', None)
df

,parameter,unit,unit_test,n,n_nan,missing_data,n_comparison,min_abs_diff,max_abs_diff,n_diff_greater_0.99,percent_diff_greater_zero
1,density_ds0,kg m-3,pass,31557,0,no missing data,2539,8.400000e-09,0.000187,0,0.0
1,practical_salinity_ds0,1,pass,31557,0,no missing data,2539,5.510000e-08,0.000141,0,0.0
1,ctdmo_seawater_pressure_ds0,dbar,pass,31557,0,no missing data,2539,2.783384e-03,0.020546,0,0.0
1,ctdmo_seawater_temperature_ds0,ºC,pass,31557,0,no missing data,2539,1.200000e-08,0.000117,0,0.0
1,ctdmo_seawater_conductivity_ds0,S m-1,pass,31557,0,no missing data,2539,1.200000e-09,0.000007,0,0.0


<a id="5" ></a>
### Observations:
- Parameters: The returned science parameters in the file are:  conductivity, temperature, pressure, salinty, and density.     
- Unit Test: Both recovered and telemetered files share the same units for the same parameters.  
- Nans: The are no nans in both files.
- Missing data: There are no data missing from the recovered data file when compared to telemetered data.  
- Value Difference: 
    - min and max absolute difference is not null but close to zero.
    - The number of data points with difference greater than 0.99 is zero. 
    - The percent of data points with difference greater than 0.99 is zero.
<a id="6" ></a>    
 ### Summary  
The data from recovered and telemetered data are consistent, which increase our confidence with the data. Some of the difference although not significant need to be addressed at the system level to try to rule out that the system when processing the data is not changing the values.

### END